# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases  Deaths    PCases  PDeaths  Cases (+)  \
0               US  30847348  556546  30539868   553268     307480   
1           Brazil  13100580  336947  12839844   325284     260736   
2            India  12801785  166177  12303131   163396     498654   
3           France   4902025   97431   4755779    96106     146246   
4           Russia   4546307   99431   4503291    97594      43016   
5   United Kingdom   4379033  127126   4364547   127006      14486   
6            Italy   3686707  111747   3607083   109847      79624   
7           Turkey   3579185   32667   3357988    31713     221197   
8            Spain   3317948   75911   3291394    75541      26554   
9          Germany   2909902   77245   2866323    76823      43579   
10        Colombia   2468236   64524   2417826    63614      50410   
11          Poland   2456709   55065   2356970    53665      99739   
12       Argentina   2428029   56634   2363251    55941      64778   
13          Mexico   2256509  204985   2244268   203664      12241   
14            Iran   1963394   63506   1897314    62759      66080   

    Deaths (+)  Fatality Rate      Continent  
0         3278            1.8  North America  
1        11663            2.6  South America  
2         2781            1.3           Asia  
3         1325            2.0         Europe  
4         1837            2.2         Europe  
5          120            2.9         Europe  
6         1900            3.0         Europe  
7          954            0.9           Asia  
8          370            2.3         Europe  
9          422            2.7         Europe  
10         910            2.6  South America  
11        1400            2.2         Europe  
12         693            2.3  South America  
13        1321            9.1  North America  
14         747            3.2           Asia

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2021-04-06 00:00:00'),
 'since': Timestamp('2021-04-01 00:00:00'),
 'Cases': 132324809,
 'Deaths': 2870511,
 'Cases (+)': 2807793,
 'Deaths (+)': 44751,
 'China Cases': 101920,
 'China Deaths': 4841,
 'China Cases (+)': 167,
 'China Deaths (+)': 0,
 'US Cases': 30847348,
 'US Deaths': 556546,
 'US Cases (+)': 307480,
 'US Deaths (+)': 3278,
 'EU Cases': 40671025,
 'EU Deaths': 930448,
 'EU Cases (+)': 800343,
 'EU Deaths (+)': 14771}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  3/28/21  3/29/21  3/30/21  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...        4       28       62   
Albania               0        0        0  ...      493      285      304   
Algeria               0        0        0  ...       86      110      115   
Andorra               0        0        0  ...       41       38       56   
Angola                0        0        0  ...       32       69       50   

                3/31/21  4/1/21  4/2/21  4/3/21  4/4/21  4/5/21  4/6/21  
Country/Region                                                           
Afghanistan          70      63      55      23      81      41      62  
Albania             434     349     336     341     348     264     141  
Algeria             131     112     125      95      98     117     140  
Andorra              66      43      62      59      57      55      42  
Angola              129      88      68     112      52      86     168  

[5 rows x 441 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)